In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

# 用tensorboard来可视化模型
from tensorboardX import SummaryWriter  # writer就相当于一个日志，保存你要做图的所有信息

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

# Hyper-parameters for updating learning rate
num_epochs = 250
learning_rate = 0.1
batch_size = 256

# lr updating parameters
epochs = [1, 3, 4, 100, 101, 180, 181, 230, 231, num_epochs]
lrs = [learning_rate, 0.2, 0.1, 0.1, 0.01, 0.01, 0.001, 0.001, 0.0001, 0.0001]

In [ ]:
class Cutout(object):
    """Randomly mask out one or more patches（补丁，就是一个遮挡小块） from an image.
    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img

def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha) # beta分布
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def CrossEntropyLoss_label_smooth(outputs, targets, device,
                                  num_classes=10, epsilon=0.075):
    N = targets.size(0)
    smoothed_labels = torch.full(size=(N, num_classes),
                                 fill_value=epsilon / (num_classes - 1))
    smoothed_labels.scatter_(dim=1, index=torch.unsqueeze(targets.to('cpu'), dim=1),
                             value=1-epsilon)
    smoothed_labels = smoothed_labels.to(device)
    log_prob = nn.functional.log_softmax(outputs, dim=1)
    loss = - torch.sum(log_prob * smoothed_labels) / N
    return loss

In [ ]:
# CIFAR-10 dataset
cifar_norm_mean = (0.49139968, 0.48215827, 0.44653124)
cifar_norm_std = (0.24703233, 0.24348505, 0.26158768)

train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True, 
                                             download=False,
                                             transform=transforms.Compose([
                                                 transforms.Pad(4),     
                                                 transforms.RandomCrop(32),       
                                                 transforms.RandomHorizontalFlip(),  
                                                 transforms.ToTensor(),  
                                                 transforms.Normalize(cifar_norm_mean, cifar_norm_std),
                                                 Cutout(n_holes=1, length=16),])
                                            )

train_dataset，val_dataset = torch.utils.data.random_split(train_dataset, [45000, 5000])

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                            train=False, 
                                            transform=transforms.Compose([
                                                transforms.ToTensor(),
                                                transforms.Normalize(cifar_norm_mean, cifar_norm_std)])                                   
                                            )

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,    
                                           shuffle=True,
                                           pin_memory=(torch.cuda.is_available()),
                                           )

val_loader = test_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False,
                                          pin_memory=(torch.cuda.is_available()),
                                          )

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False,
                                          pin_memory=(torch.cuda.is_available()),
                                          )

print(train_dataset,'\n')
print(train_dataset[0][0].shape,'\n')            
print(train_dataset[0][1],'\n')            
print(train_dataset.targets[0],'\n')      

batch=next(iter(train_loader))
images,labels = batch
print(images.shape)

In [ ]:
def Conv1(in_planes, places, stride=2): # (3,32,32)
    return nn.Sequential(
        nn.Conv2d(in_channels=in_planes,out_channels=places,kernel_size=7,stride=stride,padding=3, bias=False), #(places,16,16)
        nn.BatchNorm2d(places),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1),  # (places,8,8)
        nn.CELU(inplace=True),
    )

class Bottleneck(nn.Module):
    def __init__(self,in_places,places, stride=1,downsampling=False, expansion = 4):
        super(Bottleneck,self).__init__()
        self.expansion = expansion
        self.downsampling = downsampling

        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels=in_places,out_channels=places,kernel_size=1,stride=1, bias=False),
            nn.BatchNorm2d(places),
            nn.CELU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(places),
            nn.CELU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places*self.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(places*self.expansion),
        )

        if self.downsampling:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels=in_places, out_channels=places*self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(places*self.expansion)
            )
        self.celu = nn.CELU(inplace=True)
    def forward(self, x):
        residual = x
        out = self.bottleneck(x)

        if self.downsampling:
            residual = self.downsample(x)

        out += residual
        out = self.celu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, blocks, num_classes=1000, expansion = 4):  # (3,32,32)
        super(ResNet,self).__init__()
        self.expansion = expansion

        self.conv1 = Conv1(in_planes = 3, places= 64)  # (64,8,8)

        self.layer1 = self.make_layer(in_places = 64, places= 64, block=blocks[0], stride=1)   # (256,8,8)
        self.layer2 = self.make_layer(in_places = 256,places=128, block=blocks[1], stride=2)   # (512,4,4)
        self.layer3 = self.make_layer(in_places=512,places=256, block=blocks[2], stride=2)     # (1024,2,2)
        self.layer4 = self.make_layer(in_places=1024,places=512, block=blocks[3], stride=2)    # (2048,1,1)

        #self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(2048,num_classes)
        
        # 初始化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def make_layer(self, in_places, places, block, stride):
        layers = []
        layers.append(Bottleneck(in_places, places,stride, downsampling =True))
        for i in range(1, block):
            layers.append(Bottleneck(places*self.expansion, places))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        #x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
model = ResNet([3, 4, 6, 3], num_classes=10).to(device)

In [ ]:
# 保存模型结构
dummy_input = torch.rand(20, 3, 32, 32).to(device)  
with SummaryWriter(comment='Resnet50') as w:
    w.add_graph(model, (dummy_input,))
    
print('# model parameters:', sum(param.numel() for param in model.parameters()))

In [ ]:
# prediction function for simple data
def pred_rate(preds,labels):
    return preds.eq(labels).sum().item()/labels.shape[0]

# prediction function for data loader
def pred_rate_loader(model, val_loader, device):
    model.eval()
    with torch.no_grad():         
        correct = 0
        total = 0
        loss = []
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            loss.append(CrossEntropyLoss_label_smooth(outputs, labels, device).cpu().numpy())
            
    model.train()
    return np.mean(loss), correct / total      

# For updating learning rate
def piecewise_linear(optimizer, curr_epoch, epochs, lrs):  
    length = len(lrs)
    for i in range (length-1):
        if curr_epoch > epochs[i] and curr_epoch < epochs[i+1]:
            lr = lrs[i] + (curr_epoch-epochs[i])/(epochs[i+1]-epochs[i])*(lrs[i+1]-lrs[i])
            break
        elif curr_epoch == epochs[i]:
            lr = lrs[i]
            break
        elif curr_epoch == epochs[-1]:       
            lr = lrs[-1]
            break

    # print
    #if lrs[i] == lrs[i+1]:
        #print('Epoch [{}/{}], learning rate kept still in {}'.format(epoch+1, num_epochs, lr))
    #else:
        #print('Epoch [{}/{}], learning rate updated to {}'.format(epoch+1, num_epochs, lr))
    
    # update lr
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
    return lr

In [ ]:
# Loss and optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9, weight_decay = 5e-4, nesterov = True)

In [ ]:
# draw results
loss_ens = []
lr_ens = []
pred_rate_ens = []

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images,labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = CrossEntropyLoss_label_smooth(outputs, labels, device)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 98 == 0:
            curr_loss, curr_acc = pred_rate_loader(model, val_loader, device)
            print ("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Prediction rate: {:.4f}"
                   .format(epoch+1, num_epoches, i+1, total_step, curr_loss, curr_acc))
            
            pred_rate_ens.append(curr_acc)
            loss_ens.append(curr_loss)
            
    # update learning rate
    curr_lr = piecewise_linear(optimizer, epoch+1, epochs, lrs)
    lr_ens.append(curr_lr)

In [ ]:
# draw the loss, pred_rate with iteration andd the lr with epoch
plt. figure(figsize=(15,15))

plt.subplot(311)
plt.plot(np.arange(1,len(pred_rate_ens)+1),pred_rate_ens)
plt.title('prediction rate')
plt.xlabel('iteration')

plt.subplot(312)
plt.plot(np.arange(1,len(loss_ens)+1),loss_ens)
plt.title('loss')
plt.xlabel('iteration')

plt.subplot(313)
plt.plot(np.arange(num_epochs),lr_ens)
plt.title('learning rate')
plt.xlabel('epoch')

plt.show()

In [ ]:
# Save the model checkpoint，
torch.save(model.state_dict(), 'resnet50_piecewise_linear.ckpt')

In [ ]:
# Test the model
#model = models.resnet50().to(device)
#model.fc = nn.Linear(512, 10).to(device)  # 层都需要放到device上面

#model.load_state_dict(torch.load('resnet50_piecewise_linear.ckpt'))

model.eval()
with torch.no_grad():        
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))